# Reinforcement Learning:

Cross Entropy Method on : CartPole Environment

In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import gym
import numpy as np
import matplotlib.pyplot as plt
from tensorboardX import SummaryWriter
from collections import namedtuple

## Defining the Neural Network:

In [2]:
HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70

In [3]:
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x)

## Cross-Entropy:

In [4]:
# Episode is used to store discounted reward and a collection of episode step.
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
# EpisodeStep is used to store info about a single step - action and observation from env in an episode. We will use steps
# From Elite Episodes as training data.
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

In [5]:
def iterate_batches(env, net, batch_size):
    batch = [] # Store Episode Named Tuple.
    episode_reward = 0.0 # Stores epiode reward.
    episode_steps = [] # Stored EpisodeStep Named tuple.
    obs = env.reset()
    sm = nn.Softmax(dim=1) # Apply softmax to output to get probabilities.
    while True:
        obs_v = torch.FloatTensor([obs]) # Convert the current observation to PyTorch Tensor of size 1*4 ([obs])
        act_probs_v = sm(net(obs_v)) # Get Probabilities by passing thru NN and then applying softmax.
        act_probs = act_probs_v.data.numpy()[0] # Convert to Numpy and get probabilities.
        action = np.random.choice(len(act_probs), p=act_probs) # Randomly sample probability distributed actions to get a action
        next_obs, reward, is_done, _ = env.step(action) # Perform the action.
        episode_reward += reward # Accumulate the reward for the current episode.
        episode_steps.append(EpisodeStep(observation=obs, action=action)) # Store the observation and action taken in list.
        if is_done: # If episode Terminates.
            batch.append(Episode(reward=episode_reward, steps=episode_steps)) # Store Discounted Reward and obs and step pair.
            episode_reward = 0.0 # DO this 0 for next episode.
            episode_steps = [] # Empty for next episode.
            next_obs = env.reset() # Reset the environment.
            if len(batch) == batch_size: # If number of episodes = batch size to be passed in NN then.
                yield batch # Return Batch (Generator Function).
                batch = [] # Empty for next batch.
        obs = next_obs # if episode done then env.reset() else next state is given to obs.

In [6]:
def filter_batch(batch, percentile):
    rewards = list(map(lambda s: s.reward, batch)) # Get rewards from batch.
    reward_bound = np.percentile(rewards, percentile) # Calculate the percentile value.
    reward_mean = float(np.mean(rewards)) # Calculate the mean of the percentile.

    train_obs = []
    train_act = []
    for example in batch:
        if example.reward < reward_bound: # If reward lesser than coundary reward then do nothing.
            continue
        train_obs.extend(map(lambda step: step.observation, example.steps)) # Store observation.
        train_act.extend(map(lambda step: step.action, example.steps)) # Store action corresponding to given observation.

    train_obs_v = torch.FloatTensor(train_obs) # Convert to Tensor.
    train_act_v = torch.LongTensor(train_act)
    return train_obs_v, train_act_v, reward_bound, reward_mean

In [7]:
if __name__ == "__main__":
    env = gym.make("CartPole-v0") # Cartpole Env
    env = gym.wrappers.Monitor(env, directory="cross_entropy_CartPole", force=True) # Summary for TensorBoardX.
    obs_size = env.observation_space.shape[0] # All Observations, known environment.
    n_actions = env.action_space.n # All Actions.

    net = Net(obs_size, HIDDEN_SIZE, n_actions) # Create the model.
    objective = nn.CrossEntropyLoss() # Minimize the loss.
    optimizer = optim.Adam(params=net.parameters(), lr=0.01)
    writer = SummaryWriter(comment="-cartpole")

    # Generate bataches i.e a list of 16 episodes.
    for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)): # Continue until reward is greater than 199.
        obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE) # Filter batches generated.
        optimizer.zero_grad() # 0 the previous gradient values.
        action_scores_v = net(obs_v) # Calculate the action values.
        loss_v = objective(action_scores_v, acts_v) # Calculate the CrossEntropyLoss.
        loss_v.backward() # Caculate gradients.
        optimizer.step() # apply changes to NN weights.
        print("%d: loss=%.3f, reward_mean=%.1f, reward_bound=%.1f" % (
            iter_no, loss_v.item(), reward_m, reward_b))
        writer.add_scalar("loss", loss_v.item(), iter_no)
        writer.add_scalar("reward_bound", reward_b, iter_no)
        writer.add_scalar("reward_mean", reward_m, iter_no)
        if reward_m > 199:
            print("Solved!")
            break
    writer.close()
    env.close()

0: loss=0.695, reward_mean=21.8, reward_bound=25.0
1: loss=0.698, reward_mean=20.3, reward_bound=22.0
2: loss=0.685, reward_mean=19.4, reward_bound=21.5
3: loss=0.674, reward_mean=26.6, reward_bound=27.5
4: loss=0.666, reward_mean=28.2, reward_bound=32.0
5: loss=0.647, reward_mean=35.9, reward_bound=37.0
6: loss=0.630, reward_mean=35.9, reward_bound=41.5
7: loss=0.659, reward_mean=40.7, reward_bound=46.5
8: loss=0.639, reward_mean=49.6, reward_bound=62.0
9: loss=0.604, reward_mean=51.9, reward_bound=54.0
10: loss=0.594, reward_mean=59.3, reward_bound=59.0
11: loss=0.599, reward_mean=53.2, reward_bound=58.0
12: loss=0.605, reward_mean=49.1, reward_bound=60.5
13: loss=0.631, reward_mean=55.4, reward_bound=61.5
14: loss=0.585, reward_mean=61.6, reward_bound=64.5
15: loss=0.574, reward_mean=61.6, reward_bound=78.5
16: loss=0.572, reward_mean=54.1, reward_bound=64.0
17: loss=0.549, reward_mean=53.6, reward_bound=64.5
18: loss=0.575, reward_mean=59.8, reward_bound=69.0
19: loss=0.560, reward